In [175]:
#Jai Ganesh Deva

In [176]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize  import word_tokenize
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline,metrics, grid_search
from gensim import models, corpora
from gensim.models import ldamodel
import os
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import io
import keras.backend as K
import csv
import numpy as np
import pandas as pd
import dill as pickle
import gensim
from keras.optimizers import Adam
from gensim.models import word2vec,doc2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn import metrics

In [177]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import  Dense, Input,Conv1D,Convolution1D,Convolution2D ,LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, Masking,MaxPooling1D,GlobalMaxPool1D,Bidirectional,Merge,Flatten,Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import gc

In [178]:
os.chdir("E:\Datasets\Final Project\Politics Final\Big")

In [179]:
X_train = pd.read_csv("train_Bigdf.csv")

In [180]:
X_test = pd.read_csv("test_Bigdf.csv")

In [181]:
len(X_test)

4674

In [182]:
train_Y = np.array(X_train["sarcastic or not"])

In [183]:
test_Y = np.array(X_test["sarcastic or not"])

In [184]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
   
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
   
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [185]:
X_train["text_tokens_stop"] = X_train["text_tokens_stop"].str.replace('\d+', '')

In [186]:
X_test["text_tokens_stop"] = X_test["text_tokens_stop"].str.replace('\d+', '')

In [187]:
sentences = []

In [188]:
for sentence in X_train["text_tokens_stop"]:
    sentences.append(sentence)

In [189]:
for sentence in X_test["text_tokens_stop"]:
    sentences.append(sentence)

In [190]:
word2vec_small = word2vec.Word2Vec(sentences,sg=1,window=4,size=100,
                 min_count=2,workers=5,iter=100,sample=0.01)

In [191]:
#word2vec_small.wv.save_word2vec_format("word2vec_small")

In [202]:
tokenizer = Tokenizer(num_words=30000, lower=False, char_level=False)
tokenizer.fit_on_texts(sentences)
training_sequences = tokenizer.texts_to_sequences(X_train["lower_wp"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=200)

train_embedding_weights = np.zeros((len(train_word_index)+1, 100))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec_small[word] if word in word2vec_small else np.random.rand(100)
print(train_embedding_weights.shape)

Found 16757 unique tokens.
(16758, 100)


C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if sys.path[0] == '':
C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [203]:
test_sequences = tokenizer.texts_to_sequences(X_test["lower_wp"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=200)

In [204]:
test_cnn_data

array([[    0,     0,     0, ..., 10656, 13095, 10656],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0, 12974],
       ...,
       [    0,     0,     0, ...,     0, 13095, 10656],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0]])

In [205]:
train_cnn_data

array([[    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       ...,
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0, 10656],
       [    0,     0,     0, ...,     0,     0,     0]])

In [206]:
model = Sequential()

In [207]:

model.add(Embedding(len(train_word_index)+1,100,input_length=200,weights = [train_embedding_weights]))
model.add(LSTM(128,recurrent_dropout=0.4))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(30,name='FC1',kernel_initializer='normal',kernel_regularizer='l2'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))



In [208]:
callbacks = [EarlyStopping(monitor='val_acc', patience=10,mode='max',verbose=1)]

In [209]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 200, 100)          1675800   
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_15 (Dropout)         (None, 128)               0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512       
_________________________________________________________________
FC1 (Dense)                  (None, 30)                3870      
_________________________________________________________________
activation_15 (Activation)   (None, 30)                0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 30)                0         
__________

In [212]:
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=["accuracy"])

In [224]:
hist1 = model.fit(train_cnn_data,train_Y,epochs = 10, batch_size=32,shuffle = True)

In [ ]:
pred = model.predict_classes(test_cnn_data)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

acc = accuracy_score(test_Y,pred)
print(acc)
print(classification_report(test_Y,pred))
print(confusion_matrix(y_true = test_Y,y_pred= pred))
